In [62]:
#coding:utf-8
%reload_ext autoreload 
%autoreload 2
import os
import sys

project_basedir = '..'
sys.path.append(project_basedir)
from cchess_zero import cbf
from cchess_zero import board_visualizer
from matplotlib import pyplot as plt
import os
from cchess_zero.gameboard import *
from net import resnet
import common
from gameplays import gameplay,players
from gameplays.analysis import get_km_json
import json
%matplotlib inline

In [158]:

fullpath = '../data/validate/2018-06-28_12-00-57_833_mcts-mcts_newnet-oldnet_w.cbf'
moves = cbf.cbf2move(fullpath)
fname = fullpath.split('/')[-1]
print(fname)
state = gameplay.GameState()

2018-06-28_12-00-57_833_mcts-mcts_newnet-oldnet_w.cbf


In [159]:
from PIL import Image
import imageio
imgs_for_gif = []
state = gameplay.GameState()
statestr = 'RNBAKABNR/9/1C5C1/P1P1P1P1P/9/9/p1p1p1p1p/1c5c1/9/rnbakabnr'
for i in moves[:]: 
    statestr = GameBoard.sim_do_action(i,statestr)
    imgs_for_gif.append(board_visualizer.get_board_img(statestr))
    state.do_move(i)
imgs_for_gif = imgs_for_gif[-20:]
imageio.mimsave('../data/gif/{}-.gif'.format(fname[:-4]), imgs_for_gif,duration=0.5)

In [160]:
i

'g5i7'

In [161]:
def display_gif(fn):
    from IPython import display
    return display.HTML('<img src="{}">'.format(fn))

In [162]:
display_gif('../data/gif/{}-.gif'.format(fname[:-4]))

# ANALYSIS

In [7]:
network = resnet.get_model(os.path.join(project_basedir,
    'data/prepare_weight/2018-06-07_14-13-24'),
    common.board.create_uci_labels(),GPU_CORE=[""],FILTERS=128,NUM_RES_LAYERS=7)

0
[None, 10, 9, 128]
[None, 10, 9, 128]
[None, 10, 9, 128]
[None, 10, 9, 128]
[None, 10, 9, 128]
[None, 10, 9, 128]
[None, 10, 9, 128]
inside res net [None, 10, 9, 128]
INFO:tensorflow:Restoring parameters from ../data/prepare_weight/2018-06-07_14-13-24


In [26]:
network_player = players.NetworkPlayer('w',network,n_playout=400)

In [27]:
network_player.make_move(state=state,actual_move=False)

('f0e1', -0.6701836578454821)

In [28]:
datajson = get_km_json(network_player.mcts_policy,state.statestr,depth=40)

In [29]:
with open('../data/mstc_vis/sample.km','w') as whdl:
    json.dump(datajson,whdl)